In [330]:
#The code below sets up a tensor flow network and trains it
#Annotated code based on Aurelien Geron hands on machine learning with skitlearn and tensorflow
import tensorflow as tf
from datetime import datetime

# Common imports
import numpy as np
import os


#Sets up logdirectory indexed by the datetime
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/.format(root_logdir, now"


#Sets up the dimensions of the different layers of the network
n_inputs=300  #300 inputs, based on having run PCA to get dimensions down
n_hidden1=300 # Goes into a layer of another 300
n_hidden2=100 # Goes into a layer of 100
n_outputs=2  #2 outputs as its a binary classification problem



In [331]:
#sets up the inputs and outputs used to pass the training data to tensorflow for training

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [332]:
#function that creates a single layer of the network
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        #Obtains the number of inputs by getting the size of the second dimension i.e. the number of columns
        n_inputs = int(X.get_shape()[1])
        #Calculates 2 over the standard deviation of the inputs
        stddev = 2 / np.sqrt(n_inputs)
        #initialises the weights matrix based on the number of inputs by the number of neurons
        #using values from a truncated normal distribution
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        #Sets up the weights matrix
        W = tf.Variable(init, name="kernel")
        #A bias variable of zeros equal to the number of neurons
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        #Multiplies the input matrix by the weights matrix and adds the bias vector to the results
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [333]:
#sets up the different layers of the network and the connections between them

with tf.name_scope("dnn"):
    #The first layer takes in X as an input
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",  activation=tf.nn.relu)
    #The second hidden layer takes the first layer as an input
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    #The output layer takes the inputs from the second hidden layer and outputs it
    logits = neuron_layer(hidden2, n_outputs, name="outputs")
    

In [334]:
#Sets up the average cross entropy function that we're going to minimise
with tf.name_scope("loss"):
    print(y)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
#Writes the network definition to a log file
    
mse_summary=tf.summary.scalar('MSE',loss)
#Writes a logfile to tensor model 1
file_writer=tf.summary.FileWriter(r'filepath\tendormodel1', tf.get_default_graph())

Tensor("y_30:0", dtype=int64)


In [335]:
#The gardient descent function to minimise the optimiser that we have just defined
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #minimise the loss function
    training_op = optimizer.minimize(loss)

In [336]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

# n_epochs is the number of time you run stochastic gradient descent
n_epochs = 40
#Splits the sample into batches of size 50
batch_size = 50#

In [337]:
n_batches=(X_train.shape[0] // batch_size)


#Reads in the test data
X_test=pd.read_csv("filepath\\X_test.csv", sep=",")
y_test=pd.read_csv("filepath\\y_test.csv", sep=",")
#Converts y_test to a matrix
y_test= y_test.as_matrix()
c, r = y_test.shape 
y_test = y_test.reshape(c,)
print(y_test.shape)
#Reads in the training data
X_train=pd.read_csv("filepath\\X_train.csv", sep=",")
y_train=pd.read_csv("filepath\\y_train.csv", sep=",")



#Number of data points in the X_train dataset
m= X_train.shape[0]

#The fetch batch function sets a random sample of size batch_size from the training set
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    # random number generator over 1 to m batch_size specifies the number of random numbers returned
    indices = np.random.randint(m, size=batch_size)  # not shown
    #Returns random samples from the X_train and y_train datasets
    X_batch=X_train.ix[list(indices),:]
    y_batch=y_train.ix[list(indices)]
 
    return X_batch, y_batch





with tf.Session() as sess:
    #initialises the variables
    init.run()
    for epoch in range(n_epochs):
        #The // is a divide with integral result
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch, y_batch= fetch_batch(40,iteration,50)
            y_batch= y_batch.as_matrix()
            c, r = y_batch.shape
            y_batch = y_batch.reshape(c,) 
            #For every 10th write out the accuracy statistics
            if iteration % 10==0:
              summary_str=mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
              step=epoch*50+iteration 
              print('write')
              file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch}) 
            #Calculates the accuracy on the batch training data 
            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            #Calculates the accuracy in the test data set
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")        
    
    
file_writer.close()
           
# Then from the shell prompt run to get the tensorboard diagnostics
#tensorboard --logdir tf_logs/    
    

(3364,)
write
0 Train accuracy: 0.62 Test accuracy: 0.620987
0 Train accuracy: 0.72 Test accuracy: 0.620392
0 Train accuracy: 0.64 Test accuracy: 0.616825
0 Train accuracy: 0.64 Test accuracy: 0.615339
0 Train accuracy: 0.68 Test accuracy: 0.615339
0 Train accuracy: 0.6 Test accuracy: 0.616825
0 Train accuracy: 0.66 Test accuracy: 0.612366
0 Train accuracy: 0.62 Test accuracy: 0.616231
0 Train accuracy: 0.46 Test accuracy: 0.615042
0 Train accuracy: 0.72 Test accuracy: 0.615042
write
0 Train accuracy: 0.68 Test accuracy: 0.615042
0 Train accuracy: 0.66 Test accuracy: 0.615042
0 Train accuracy: 0.8 Test accuracy: 0.620095
0 Train accuracy: 0.58 Test accuracy: 0.620095
0 Train accuracy: 0.6 Test accuracy: 0.620095
0 Train accuracy: 0.66 Test accuracy: 0.620095
0 Train accuracy: 0.7 Test accuracy: 0.620095
0 Train accuracy: 0.64 Test accuracy: 0.62069
0 Train accuracy: 0.6 Test accuracy: 0.62069
0 Train accuracy: 0.74 Test accuracy: 0.62069
write
0 Train accuracy: 0.58 Test accuracy: 0.62